In [2]:
from main.paths import ROOT_PATH  # isort:skip

In [3]:
import argparse
import os.path as osp
import sys

In [4]:
import cv2
from loguru import logger

import torch

import random
import os
import numpy as np

In [5]:
from videoanalyst.config.config import cfg as root_cfg
from videoanalyst.config.config import specify_task
from videoanalyst.data import builder as dataloader_builder
from videoanalyst.engine import builder as engine_builder
from videoanalyst.model import builder as model_builder
from videoanalyst.optim import builder as optim_builder
from videoanalyst.utils import Timer, complete_path_wt_root_in_cfg, ensure_dir
from videoanalyst.pipeline import builder as pipeline_builder
from videoanalyst.engine.builder import build as tester_builder
from videoanalyst.engine.monitor.monitor_impl.tensorboard_logger import TensorboardLogger

In [6]:
cv2.setNumThreads(1)


In [7]:
# torch.backends.cudnn.enabled = False

# pytorch reproducibility
# https://pytorch.org/docs/stable/notes/randomness.html#cudnn
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [8]:
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [14]:
def make_parser():
    parser = argparse.ArgumentParser(description='Test') # Train?
    parser.add_argument('-cfg',
                        '--config',
                        default='experiments/stmtrack/train/got10k/stmtrack-googlenet-trn-edited.yaml',
                        # experiments/stmtrack/train/fulldata/stmtrack-googlenet-trn-fulldata.yaml
                        type=str,
                        help='path to experiment configuration')
    parser.add_argument(
        '-r',
        '--resume',
        default="",
        help=r"completed epoch's number, latest or one model path")
    parser.add_argument(
        '-v',
        '--validation',
        default="15",
        help=r"Epoch's number to start to evaluate the model on the validation set")

    return parser

# main

In [15]:
set_seed(1000000007)

In [16]:
parser = make_parser()

In [20]:
# parsed_args = parser.parse_args()

In [24]:
parsed_args_config = 'experiments/stmtrack/train/got10k/stmtrack-googlenet-trn-edited.yaml'
# experiments/stmtrack/train/fulldata/stmtrack-googlenet-trn-fulldata.yaml

In [25]:
exp_cfg_path = osp.realpath(parsed_args_config)

In [26]:
root_cfg.merge_from_file(exp_cfg_path)

In [27]:
root_cfg = complete_path_wt_root_in_cfg(root_cfg, ROOT_PATH)
# root_cfg = root_cfg.train

In [28]:
task, task_cfg = specify_task(root_cfg.train)

In [32]:
task_cfg.freeze()

In [33]:
log_dir = osp.join(task_cfg.exp_save, task_cfg.exp_name, "logs")

In [34]:
ensure_dir(log_dir)

In [35]:
logger.configure(
        handlers=[
            dict(sink=sys.stderr, level="INFO"),
            dict(sink=osp.join(log_dir, "train_log.txt"),
                 enqueue=True,
                 serialize=True,
                 diagnose=True,
                 backtrace=True,
                 level="INFO")
        ],
        extra={"common_to_all": "default"},
    )

[1, 2]

In [37]:
# backup config
logger.info("Load experiment configuration at: %s" % exp_cfg_path)
logger.info(
    "Merged with root_cfg imported from videoanalyst.config.config.cfg")
cfg_bak_file = osp.join(log_dir, "%s_bak.yaml" % task_cfg.exp_name)

2024-08-12 10:15:14.119 | INFO     | __main__:<module>:2 - Load experiment configuration at: /home/meysam/test-apps/STMTrack/experiments/stmtrack/train/got10k/stmtrack-googlenet-trn-edited.yaml
2024-08-12 10:15:14.120 | INFO     | __main__:<module>:3 - Merged with root_cfg imported from videoanalyst.config.config.cfg
